In [1]:
#!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

In [2]:
#!pip install tensorboardX
#!pip install scikit-image

In [3]:
import os
import time
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from tensorboardX import SummaryWriter

from utils import secondsToHM

In [5]:
t = Trainer()

cuda


/home/mp6021/.local/lib/python3.8/site-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [ ]:
t.train()

Total Trainable Parameters : 36867202
Total Steps : 33160
Training --- Epoch : 0


/home/mp6021/.local/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
/home/mp6021/.local/lib/python3.8/site-packages/torch/nn/functional.py:4193: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


Epoch :   0 | Batch :       0, examples/s:  25.3, loss : 0.15857, time elapsed: 00h00m14s, time left: 00h00m00s
Epoch :   0 | Batch :     200, examples/s:  28.3, loss : 0.13839, time elapsed: 00h04m41s, time left: 12h53m02s
Epoch :   0 | Batch :     400, examples/s:  28.4, loss : 0.14211, time elapsed: 00h09m04s, time left: 12h23m23s
Epoch :   0 | Batch :     600, examples/s:  28.4, loss : 0.13533, time elapsed: 00h13m37s, time left: 12h19m10s
Epoch :   0 | Batch :     800, examples/s:  28.3, loss : 0.13588, time elapsed: 00h18m02s, time left: 12h09m29s
Epoch :   0 | Batch :    1000, examples/s:  28.3, loss : 0.12581, time elapsed: 00h22m27s, time left: 12h02m22s
Epoch :   0 | Batch :    1200, examples/s:  28.3, loss : 0.12823, time elapsed: 00h26m48s, time left: 11h53m52s
Epoch :   0 | Batch :    1400, examples/s:  28.3, loss : 0.13625, time elapsed: 00h31m13s, time left: 11h48m21s
Epoch :   0 | Batch :    1600, examples/s:  28.3, loss : 0.12785, time elapsed: 00h35m42s, time left: 11